In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import os
from time import sleep

In [2]:
headers = {'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
'accept-encoding':'gzip, deflate, br, zstd',
'accept-language':'ko,en;q=0.9,en-US;q=0.8',
'priority':'u=0, i',
'sec-ch-ua':'"Not;A=Brand";v="99", "Microsoft Edge";v="139", "Chromium";v="139"',
'sec-ch-ua-mobile':'?0',
'sec-ch-ua-platform':'"Windows"',
'sec-fetch-dest':'document',
'sec-fetch-mode':'navigate',
'sec-fetch-site':'none',
'sec-fetch-user':'?1',
'upgrade-insecure-requests':'1',
'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36 Edg/139.0.0.0'}

In [3]:
from datetime import datetime, timedelta

def date_range(start_date: str, end_date: str):
    # 문자열을 datetime 객체로 변환
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    # 날짜 리스트 생성
    dates = []
    current = start
    while current <= end:
        dates.append(current.strftime("%Y-%m-%d"))
        current += timedelta(days=1)
    
    return dates

page_urls = [f'https://news.hada.io/past?day={day}' for day in date_range("2023-01-01", "2025-08-25")]

In [4]:

origin_url = 'https://news.hada.io/'

In [ ]:
data = []
pbar = tqdm(total=len(page_urls))
for url in page_urls:
    pbar.update(1)
    req = requests.get(url,headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    for one in soup.select("div[class='topics'] > div[class='topic_row']"):
        title = one.select_one("div[class='topictitle'] > a").text.strip()
        try:
            ellipsis = one.select_one("div[class='topicdesc'] > a").text.strip()
            href = origin_url+one.select_one("div[class='topicdesc'] > a")['href']
            source_link = one.select_one("div[class='topictitle'] > a")['href']
            data.append(dict(title=title, ellipsis=ellipsis, href=href, source_link=source_link))
        except:
            pass
pbar.close()
data

100%|██████████| 968/968 [01:20<00:00, 12.00it/s]


[{'title': '미리보는 CES 2023 [27p 슬라이드]',
  'ellipsis': "- 주제 : '기술혁신이 인류의 문제를 해결하는 방향과 지속가능성'  \r\n- 기조연설 : AMD, BMW, Teladoc, Nasdaq, Delta, Samsung, Wing,..  \r\n- 웹...",
  'href': 'https://news.hada.io/topic?id=8150',
  'source_link': 'https://drive.google.com/file/d/1ctlVPJLjFrT6FM3xrK51xA26xvupItC_/view'},
 {'title': 'GPT3Discord - 디스코드 내에서 GPT3사용하게 해주는 봇',
  'ellipsis': '- `!g `: 바로 GPT3 프롬프트 입력  \r\n- `!g converse`: ChatGPT처럼 대화 하기  \r\n  - 채팅이 길어져서 대화한계에 도달하면, 봇이 GPT3를 이용해...',
  'href': 'https://news.hada.io/topic?id=8151',
  'source_link': 'https://github.com/Kav-K/GPT3Discord'},
 {'title': 'ESP32 아두이노를 위한 WireGuard 구현',
  'ellipsis': '- lwIP(Lightweight IP) Stack용으로 구현된 WireGuard를 아두이노에 맞게 조정   \r\n- lwIP WireGuard의 특징   \r\n  - malloc을 사용하지 않아서 ...',
  'href': 'https://news.hada.io/topic?id=8152',
  'source_link': 'https://github.com/ciniml/WireGuard-ESP32-Arduino'},
 {'title': '가장 인상적인 1인 회사들',
  'ellipsis': '1인이 운영하는 회사 Top 10(수익면에서). 그들이 어떻게 회사를 키웠는지 정리한 링크도 포함   \r\n# 1. 

In [9]:
df_page_data = pd.DataFrame(data)

In [ ]:
# os.makedirs('data', exist_ok=True)
# df_page_data.to_csv('data/page_data.csv', index=False)

In [4]:
df_page_data = pd.read_csv('data/page_data.csv')
df_page_data.head()

,title,ellipsis,href,source_link
0,미리보는 CES 2023 [27p 슬라이드],- 주제 : '기술혁신이 인류의 문제를 해결하는 방향과 지속가능성' \r\n- 기...,https://news.hada.io/topic?id=8150,https://drive.google.com/file/d/1ctlVPJLjFrT6F...
1,GPT3Discord - 디스코드 내에서 GPT3사용하게 해주는 봇,- `!g `: 바로 GPT3 프롬프트 입력 \r\n- `!g converse`:...,https://news.hada.io/topic?id=8151,https://github.com/Kav-K/GPT3Discord
2,ESP32 아두이노를 위한 WireGuard 구현,- lwIP(Lightweight IP) Stack용으로 구현된 WireGuard를...,https://news.hada.io/topic?id=8152,https://github.com/ciniml/WireGuard-ESP32-Arduino
3,가장 인상적인 1인 회사들,1인이 운영하는 회사 Top 10(수익면에서). 그들이 어떻게 회사를 키웠는지 정리...,https://news.hada.io/topic?id=8157,https://hackernoon.com/check-out-these-impress...
4,TypeScript 배우기에 좋은 오픈소스 프로젝트는?,HN에 올라온 질문의 답들 \r\n* Excalidraw https://gith...,https://news.hada.io/topic?id=8155,https://news.ycombinator.com/item?id=34205294


In [5]:
data = []
pbar = tqdm(total=len(df_page_data))
for idx,url in enumerate(df_page_data['href']):
    switch = True
    while switch:
        try:
            req = requests.get(url,headers=headers)
            soup = BeautifulSoup(req.text, 'html.parser')
            content = soup.select_one("div[class='topic_contents']").text.strip()
            date = soup.select_one("div[class='topicinfo'] > span[title]")['title']
            data.append(dict(href=url,content=content,date=date))
        except:
            sleep(5)
        switch = False
        if idx % 100 == 0:
            sleep(5)
    pbar.update(1)
pbar.close()


  0%|          | 0/11462 [00:00<?, ?it/s]

100%|██████████| 11462/11462 [27:33<00:00,  6.93it/s] 


In [8]:
df = pd.DataFrame(data)
geeknews_df = pd.merge(df_page_data,df,on='href',how='left')
geeknews_df.to_csv('data/geeknews.csv',index=False)

In [10]:
geeknews_df

,title,ellipsis,href,source_link,content,date
0,미리보는 CES 2023 [27p 슬라이드],- 주제 : '기술혁신이 인류의 문제를 해결하는 방향과 지속가능성' \r\n- 기...,https://news.hada.io/topic?id=8150,https://drive.google.com/file/d/1ctlVPJLjFrT6F...,주제 : '기술혁신이 인류의 문제를 해결하는 방향과 지속가능성'\n기조연설 : AM...,2023-01-01 10:06
1,GPT3Discord - 디스코드 내에서 GPT3사용하게 해주는 봇,- `!g `: 바로 GPT3 프롬프트 입력 \r\n- `!g converse`:...,https://news.hada.io/topic?id=8151,https://github.com/Kav-K/GPT3Discord,!g <prompt>: 바로 GPT3 프롬프트 입력\n\n!g converse: C...,2023-01-01 10:20
2,ESP32 아두이노를 위한 WireGuard 구현,- lwIP(Lightweight IP) Stack용으로 구현된 WireGuard를...,https://news.hada.io/topic?id=8152,https://github.com/ciniml/WireGuard-ESP32-Arduino,lwIP(Lightweight IP) Stack용으로 구현된 WireGuard를 아...,2023-01-01 10:37
3,가장 인상적인 1인 회사들,1인이 운영하는 회사 Top 10(수익면에서). 그들이 어떻게 회사를 키웠는지 정리...,https://news.hada.io/topic?id=8157,https://hackernoon.com/check-out-these-impress...,1인이 운영하는 회사 Top 10(수익면에서). 그들이 어떻게 회사를 키웠는지 정리...,2023-01-02 11:37
4,TypeScript 배우기에 좋은 오픈소스 프로젝트는?,HN에 올라온 질문의 답들 \r\n* Excalidraw https://gith...,https://news.hada.io/topic?id=8155,https://news.ycombinator.com/item?id=34205294,HN에 올라온 질문의 답들\n\nExcalidraw https://github.co...,2023-01-02 10:19
...,...,...,...,...,...,...
11457,Nitro: 작고 유연한 init 시스템 및 프로세스 슈퍼바이저,"- Nitro는 임베디드, 서버, 데스크톱, 컨테이너에 모두 적용 가능한 초소형 프...",https://news.hada.io/topic?id=22691,https://git.vuxu.org/nitro/about/,"Nitro는 임베디드, 서버, 데스크톱, 컨테이너에 모두 적용 가능한 초소형 프로세...",2025-08-24 10:01
11458,모든 것은 서로 상관관계가 있음 (2014–23),- 사회과학 및 생명과학에서는 거의 모든 변수 간에 일정 수준의 상관관계가 존재함\...,https://news.hada.io/topic?id=22689,https://gwern.net/everything,사회과학 및 생명과학에서는 거의 모든 변수 간에 일정 수준의 상관관계가 존재함\n이...,2025-08-24 09:55
11459,휴대폰으로 쇼핑카트 바퀴를 제어하기 (2021),- Gatekeeper Systems 바퀴를 잠그거나 잠금 해제하려면 휴대폰 스피커...,https://news.hada.io/topic?id=22688,https://www.begaydocrime.com/,Gatekeeper Systems 바퀴를 잠그거나 잠금 해제하려면 휴대폰 스피커로 ...,2025-08-24 09:52
11460,"모든 관리자는 실수를 하지만, 좋은 관리자는 인정하고 고침",- 관리자가 되면 실수를 피할 수 없음을 강조함\n- 중요한 관리 기술로서 실수 이...,https://news.hada.io/topic?id=22686,https://terriblesoftware.org/2025/08/22/the-ma...,관리자가 되면 실수를 피할 수 없음을 강조함\n\n중요한 관리 기술로서 실수 이후의...,2025-08-24 09:46
